## Import required libraries

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

## Cleaning, Feature Engineering and Merging
In this notebook, we will clean the data for 4 tables, as follows:

- Uring Drug Screen - Scheduled urine drug screen taken once per week for 24 weeks for 8 different drug classes
- Self Reported Use - Scheduled survey collection every 4 weeks, includes self reported drug use from the previous 4 weeks
- Medication Doses - Patients received methadone or buprenorphine, dosed daily for 24 weeks
- Demographics - Includes pateints sex, ethnicity and race
- RSA - Research session attendance - records attendance for each patient for 24 weeks

After each table is cleaned, we will create features to improve the data quality and then merge all tables to create a high quality dataset to feed into a machine learning model to make predictions.

## Helper Functions
Below are some reusable functions listed as follows:

- Clean Dataframes - Annotates columns, drops erroneous columns and transforms appropriate data types where required<br><br>
- Backfill Nulls - There is missing data that results from human error.  We will fill these in with the previous rows datapoint for imputation strategy for certain features<br><br>
- Aggregate columns - For some clinical data, there are multiple rows per patient.  We will reduce rows with aggregation to improve data quality.<br><br>
- Flatten Dataframes - This function will reshape and structure the data so that each patient is reflected as a single row, with all clinical data in properly encoded columns<br><br>
- Merge Dataframes - Once all dataframes are transformed, they will be merged and processed for machine learning


## Clean Dataframes
Will apply appropriate cleaning tasks

In [2]:
def clean_df(df, keep_cols, rename_cols):
    """
    Clean the given DataFrame by dropping unnecessary columns, renaming columns, and reordering columns.

    Parameters:
    df (pandas.DataFrame): The DataFrame to be cleaned.
    keep_cols (list): A list of column names to keep in the DataFrame.
    rename_cols (dict): A dictionary mapping old column names to new column names.

    Returns:
    pandas.DataFrame: The cleaned DataFrame.
    """
    # drop columns that are not on keep_cols list
    df = df.drop(columns=[col for col in df.columns if col not in keep_cols])
    
    # cleans the VISIT column, removing text and converting to integers for ordinal value
    if 'VISIT' in df.columns:
        # remove 'VISIT' from VISIT column
        df['VISIT'] = df['VISIT'].str.replace('VISIT','')
        
        # if VISIT column contains 'BASELINE' replace with 0
        df['VISIT'] = df['VISIT'].str.replace('BASELINE','0')

        # remove WK in visit column then convert to int
        df['VISIT'] = df['VISIT'].str.replace('WK','')
        
        # change VISIT column to int
        df['VISIT'] = df['VISIT'].astype(int)
    else:
        pass

    
    # rename columns
    df = df.rename(columns=rename_cols)

    # bring the last column to first position as it will serve as the primary key column for all dfs
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]


    return df


## Backfill Nulls
In columns for medication and total dose, there are missing values from human error in data entry

In [3]:
def backfill_nulls(df, cols):
    """
    Backfill null values in the given columns with the last non-null value.

    Parameters:
    df (pandas.DataFrame): The DataFrame to be cleaned.
    cols (list): A list of column names to backfill.

    Returns:
    pandas.DataFrame: The cleaned DataFrame.
    """
    for col in cols:
        df[col] = df[col].fillna(method='bfill')
    return df

## Aggregate columns
We will aggregate data stored in multiple rows to improve data quality.

In [4]:
def agg_df(df, index, agg):
    """
    Aggregate the given DataFrame by grouping by the given index and aggregating the given columns.

    Parameters:
    df (pandas.DataFrame): The DataFrame to be cleaned.
    index (list): A list of column names to group by.
    agg (dict): A dictionary mapping column names to aggregation functions.

    Returns:
    pandas.DataFrame: The aggregated DataFrame.
    """
    df = df.groupby(index).agg(agg)
    df = df.reset_index()
    return df   

## Flatten Dataframes
- This function will create a new DF for each week of treatment
- In each dataframe, the columns will be encoded to reflect clinical data for that week of treatment
- Each DF will then be merged to final dataset


In [5]:
def flatten_dataframe(df,start,stop,step):
    """
    Flattens a dataframe by creating separate dataframes for each week of clinical data,
    renaming columns with the corresponding week number, and merging all dataframes into one,
    reshaping dataframe to 1 row per patient, with all clinical data properly encoded into columns.

    Args:
        df (pandas.DataFrame): The input dataframe.
        start (int): The starting week number.
        stop (int): The stopping week number.
        step (int): The step size between weeks.

    Returns:
        pandas.DataFrame: The flattened dataframe.

    """
    # create a new dataframe for every week of clinical data
    # the name of the dataframe will be VISIT+number of visit
    for i in range(start,stop+1,step):
        globals()['VISIT%s' % i] = df[df['VISIT']==i]

    # for each dataframe beteween start and stop
    # add the value in VISIT to the end of the name of each column +"_"+"visit"
    for i in range(start,stop+1,step):
        for col in globals()['VISIT%s' % i].columns:
            if col != 'patdeid':
                globals()['VISIT%s' % i][col+'_'+str(i)] = globals()['VISIT%s' % i][col]
                # after columns are annoted, drop original columns            
                globals()['VISIT%s' % i] = globals()['VISIT%s' % i].drop(columns=col)
            else:
                pass

    # merge all dfs using left merge on patdeid
    for i in range(start,stop+1,step):
        if i == start:
            df = pd.merge(globals()['VISIT%s' % i], globals()['VISIT%s' % (i+step)], on='patdeid', how='left')
        elif i < stop:
            df = pd.merge(df, globals()['VISIT%s' % (i+step)], on='patdeid', how='left')
        else:
            pass

            # drop erroneous visit columns, as the visit is encoded in each column
            df = df.drop(columns=[col for col in df.columns if col.startswith('VISIT')], axis=1)
            

            return df

## Merge Dataframes
- This function will merge a group of dataframes with common keys
- The dataframes must be stored on a list and will be treated as one iterable

In [6]:
# create function to merge dataframes using functools reduce
def merge_dfs(dfs):
    """
    Merge the given list of DataFrames into one DataFrame.

    Parameters:
    dfs (list): A list of DataFrames to be merged.

    Returns:
    pandas.DataFrame: The merged DataFrame.
    """
    from functools import reduce
    df = reduce(lambda left,right: pd.merge(left,right,on='patdeid'), dfs)
    return df

## Import the unprocessed CSV files


In [7]:
# medication doses
med = pd.read_csv('../data/raw_data_files/T_FRDOS.csv')

# urine drug screens
uds = pd.read_csv('../data/raw_data_files/T_FRUDSAB.csv')

# self reported drug use
sru = pd.read_csv('../data/raw_data_files/T_FRTFB.csv')

# patient demographics
demo = pd.read_csv('../data/raw_data_files/T_FRDEM.csv')

# research session attendance (RSA)
rsa = pd.read_csv('../data/raw_data_files/T_FRRSA.csv')

## Transform the medication doses data

In [8]:
# define parameters
med_cols = ['patdeid','VISIT','DOS002','DOS005','DOS006']
rename_cols = {'DOS002':'medication','DOS005':'total_dose','DOS006':'admin_location'}

# execute clean function
med = clean_df(med, med_cols, rename_cols)

med

,patdeid,VISIT,medication,total_dose,admin_location
0,1,0,2.0,8.0,1.0
1,1,1,2.0,16.0,1.0
2,1,1,2.0,24.0,1.0
3,1,1,2.0,24.0,1.0
4,1,1,2.0,32.0,1.0
...,...,...,...,...,...
160903,1931,24,2.0,8.0,1.0
160904,1931,24,2.0,8.0,1.0
160905,1931,24,2.0,8.0,1.0
160906,1931,24,2.0,8.0,1.0


## Backfill Nulls
There is missing data that comes from human error, for the medication doses and admin location.  We will backfill those nulls with the previous rows data to maintain accuracy.

In [9]:
# define paramteres
back_fill_cols = ['total_dose','medication','admin_location']

# execute backfill function
med = backfill_nulls(med, back_fill_cols)

# observe backfill columns in context with patient and visit data for first 25 rows
med.loc[:,['patdeid','VISIT','total_dose','medication','admin_location']][:25] 

,patdeid,VISIT,total_dose,medication,admin_location
0,1,0,8.0,2.0,1.0
1,1,1,16.0,2.0,1.0
2,1,1,24.0,2.0,1.0
3,1,1,24.0,2.0,1.0
4,1,1,32.0,2.0,1.0
5,1,1,32.0,2.0,1.0
6,1,1,32.0,2.0,1.0
7,1,2,32.0,2.0,1.0
8,1,2,32.0,2.0,1.0
9,1,2,32.0,2.0,1.0


## Aggregate columns
Data for medication doses was recorded daily, adding multiple rows per patient.  We will consolidate the data and aggregate total dose on a weekly basis, to improve data quality.

In [10]:
# define parameters
index_columns = ['patdeid','VISIT']

# dictionary with columsn to aggregate and aggregation functions
agg_dict = {'total_dose':'sum','medication':'first','admin_location':'first'}

# execute aggregation
med = agg_df(med, index_columns, agg_dict)

print(f'New shape for aggregated dataframe, reduced from 106,000 rows to {med.shape[0]}')
med[:5]

New shape for aggregated dataframe, reduced from 106,000 rows to 23528


,patdeid,VISIT,total_dose,medication,admin_location
0,1,0,8.0,2.0,1.0
1,1,1,160.0,2.0,1.0
2,1,2,320.0,2.0,1.0
3,1,3,192.0,2.0,1.0
4,1,4,384.0,2.0,1.0


### Create df for medication type
We will create a separate dataframe for medication type; we will merge this later, which will help with applying filters for reporting

In [11]:
# create function to create df for medication column 
medication = med[['patdeid','medication']].drop_duplicates(subset='patdeid').reset_index(drop=True)

print(f'Medication dataframe shape{medication.shape}')
medication[:5]

Medication dataframe shape(1315, 2)


,patdeid,medication
0,1,2.0
1,2,2.0
2,3,1.0
3,4,2.0
4,6,2.0


## Feature Engineering
To improve data quality, we will segment the weekly medication doses into separate columns by medication (methadone or buprenorphine)


In [12]:
def med_features(df):
    """
    Process the medication dataframe by creating separate columns for methadone dose and buprenorphine dose,
    filling null values with 0, and dropping unnecessary columns.

    Parameters:
    df (pandas.DataFrame): The medication dataframe.

    Returns:
    pandas.DataFrame: The processed dataframe.
    """
    # creeate new columns for methadone and buprenorphine dose
    df['methadone_dose'] = df.loc[df.medication==1.0]['total_dose']
    df['bupe_dose'] = df.loc[df.medication==2.0]['total_dose']

    # fill null values with 0
    df.methadone_dose.fillna(0, inplace=True)
    df.bupe_dose.fillna(0, inplace=True)

    # drop original columns to remove redundancy
    df = df.drop(columns=['total_dose','medication'])

    return df


In [13]:
med = med_features(med)

med[:5]

,patdeid,VISIT,admin_location,methadone_dose,bupe_dose
0,1,0,1.0,0.0,8.0
1,1,1,1.0,0.0,160.0
2,1,2,1.0,0.0,320.0
3,1,3,1.0,0.0,192.0
4,1,4,1.0,0.0,384.0


### Feature engineering
We will create a separate dataframe to track treatment retention.  The function in the next cell will create a new column `total_visits`, counting the number of weeks each patient received treatment.  We will store the data in a separate dataframe and merge later for appropriate processing.

In [14]:
# create function to remove total visits column, while dropping duplicates and keeping patdeid as subset
def treatment_retention(df):
    
    # track treatment retention counting number of visits for each patient
    df['total_visits'] = df.groupby('patdeid')['VISIT'].transform('nunique')

    # save total visits column to new dataframe with patdeid column
    total_visits = df[['patdeid','total_visits']].drop_duplicates(subset='patdeid').reset_index(drop=True)

    # drop total visits column from original dataframe 
    df = df.drop(columns='total_visits')

    return df, total_visits
    

In [15]:
# execute treatment retention funciton
med, total_visits = treatment_retention(med)

print('Confirm that the total_visits columns was droped')
display(med[:5]) # show first 5 rows of med df
print(f'Shape for total_vists dataframe: {total_visits.shape}')
display(total_visits[:5]) # show first 5 rows of total_visits df

Confirm that the total_visits columns was droped


,patdeid,VISIT,admin_location,methadone_dose,bupe_dose
0,1,0,1.0,0.0,8.0
1,1,1,1.0,0.0,160.0
2,1,2,1.0,0.0,320.0
3,1,3,1.0,0.0,192.0
4,1,4,1.0,0.0,384.0


Shape for total_vists dataframe: (1315, 2)


,patdeid,total_visits
0,1,25
1,2,25
2,3,25
3,4,25
4,6,3


## Flatten dataframe for machine learning
This function will create a column for each week and medication dose of treatment

In [16]:
# parameters for merge function
start = 0 # zero represents the assessment visit, where clinical data tracking begins
stop = 24 # the stop period is the end of treatment
step = 1 # transform each week of clinical data into a separate dataframe

# execute flatten function
med = flatten_dataframe(med,start,stop,step)

# observe shape and first 5 rows of flattened dataframe
print(f'Shape of flattened dataframe: {med.shape}')
med[:5]

Shape of flattened dataframe: (1314, 76)


,patdeid,admin_location_0,methadone_dose_0,bupe_dose_0,admin_location_1,methadone_dose_1,bupe_dose_1,admin_location_2,methadone_dose_2,bupe_dose_2,...,bupe_dose_21,admin_location_22,methadone_dose_22,bupe_dose_22,admin_location_23,methadone_dose_23,bupe_dose_23,admin_location_24,methadone_dose_24,bupe_dose_24
0,1,1.0,0.0,8.0,1.0,0.0,160.0,1.0,0.0,320.0,...,180.0,2.0,0.0,246.0,1.0,0.0,128.0,1.0,0.0,166.0
1,2,1.0,0.0,8.0,2.0,0.0,48.0,1.0,0.0,48.0,...,72.0,1.0,0.0,60.0,1.0,0.0,72.0,1.0,0.0,68.0
2,3,1.0,30.0,0.0,1.0,170.0,0.0,1.0,310.0,0.0,...,0.0,1.0,630.0,0.0,1.0,510.0,0.0,1.0,540.0,0.0
3,4,1.0,0.0,16.0,1.0,0.0,152.0,1.0,0.0,192.0,...,32.0,1.0,0.0,160.0,1.0,0.0,128.0,1.0,0.0,32.0
4,6,1.0,0.0,16.0,1.0,0.0,8.0,1.0,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# observe columns in long format
list(med.columns)

['patdeid',
 'admin_location_0',
 'methadone_dose_0',
 'bupe_dose_0',
 'admin_location_1',
 'methadone_dose_1',
 'bupe_dose_1',
 'admin_location_2',
 'methadone_dose_2',
 'bupe_dose_2',
 'admin_location_3',
 'methadone_dose_3',
 'bupe_dose_3',
 'admin_location_4',
 'methadone_dose_4',
 'bupe_dose_4',
 'admin_location_5',
 'methadone_dose_5',
 'bupe_dose_5',
 'admin_location_6',
 'methadone_dose_6',
 'bupe_dose_6',
 'admin_location_7',
 'methadone_dose_7',
 'bupe_dose_7',
 'admin_location_8',
 'methadone_dose_8',
 'bupe_dose_8',
 'admin_location_9',
 'methadone_dose_9',
 'bupe_dose_9',
 'admin_location_10',
 'methadone_dose_10',
 'bupe_dose_10',
 'admin_location_11',
 'methadone_dose_11',
 'bupe_dose_11',
 'admin_location_12',
 'methadone_dose_12',
 'bupe_dose_12',
 'admin_location_13',
 'methadone_dose_13',
 'bupe_dose_13',
 'admin_location_14',
 'methadone_dose_14',
 'bupe_dose_14',
 'admin_location_15',
 'methadone_dose_15',
 'bupe_dose_15',
 'admin_location_16',
 'methadone_dose_16'

### Medication table properly transformed

## Transform the urine drug screen data

In [18]:
# define parameters
uds_cols = ['patdeid','VISIT','AB001','AB001B','UDS001','UDS001A','UDS001B', 'UDS003', 'UDS004', 'UDS005', 'UDS006', 'UDS007', 'UDS008', 'UDS009', 'UDS010', 'UDS011', 'UDS012', 'UDS013']

rename_cols = {'AB001':'t_alcohol','AB001B':'alc_result','UDS001':'urine_test','UDS001A':'refuse_reason','UDS001B':'other_reason', 'UDS003':'urine_temp','UDS004':'supervised','UDS005':'t_Amphetamines', 'UDS006':'t_Benzodiazepines','UDS007':'t_Methadone', 'UDS008':'t_Oxycodone', 'UDS009':'t_Cocaine', 'UDS010':'t_Methamphetamine', 'UDS011':'t_Opiate300', 'UDS012':'t_Cannabinoids', 'UDS013':'t_Propoxyphene'}

# execute clean function
uds = clean_df(uds, uds_cols, rename_cols)

# observe shape and first 5 rows of cleaned dataframe
print(f'Shape of cleaned dataframe: {uds.shape}')
uds[:5]

Shape of cleaned dataframe: (24930, 18)


,patdeid,VISIT,t_Propoxyphene,t_Amphetamines,t_alcohol,alc_result,urine_test,refuse_reason,other_reason,urine_temp,supervised,t_Cannabinoids,t_Benzodiazepines,t_Methadone,t_Oxycodone,t_Cocaine,t_Methamphetamine,t_Opiate300
0,1,0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,0.0,1.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Feature engineering
Within the urine drug screen data, will will define a target outcome<br>
- The target outcome will be an abstinence window, which is the final 4 weeks of treatement
- If a patient shows 4 consectuive clean urine tests for opiates at the final 4 weeks of treatment, the patient has reached a successful treatment outcome


In [19]:
def create_outcome_column(df):
    """
    Create a column for the target outcome based on the urine drug screen data.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the urine drug screen data.

    Returns:
    pandas.DataFrame: The DataFrame with the outcome column added.
    """
    # Create column for target
    df['outcome'] = None

    # Look for unique patient id in patdeid column
    for i in df['patdeid'].unique():

        # if patient ID and VISIT are in the 21,22,23,34 weeks and optiate tests are 0
        if (df[(df['patdeid']==i) & (df['VISIT'].isin([21,22,23,24])) & (df['t_Opiate300']==0)].shape[0] == 4):

            # For each patitent ID at the row for VISIT 0, set outcome to 1
            df.loc[(df['patdeid']==i) & (df.VISIT==0), 'outcome'] = 1.0

        else:
            # For each patitent ID at the row for VISIT 0, set outcome to 0
            df.loc[(df['patdeid']==i) & (df.VISIT==0), 'outcome'] = 0.0

    # Replace NaN with 0
    df['outcome'] = df['outcome'].replace(np.nan, 0.0)

    return df


In [20]:
# execute target variable function
uds = create_outcome_column(uds)

# review shape and first 5 rows
print(f'Shape of dataframe: {uds.shape}')
uds[:5]

Shape of dataframe: (24930, 19)


,patdeid,VISIT,t_Propoxyphene,t_Amphetamines,t_alcohol,alc_result,urine_test,refuse_reason,other_reason,urine_temp,supervised,t_Cannabinoids,t_Benzodiazepines,t_Methadone,t_Oxycodone,t_Cocaine,t_Methamphetamine,t_Opiate300,outcome
0,1,0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1,1,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,0.0,1.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


We will create a view to validate that the outcome column reflects that the patient successfully reached the abstinence window

In [21]:
#  function to produce random number from patdeid column
def random(df):
    return np.random.choice(uds.patdeid)
    
# run this cell multiple times to see random patient data to see if outcome column is accurate
uds.loc[uds.patdeid==random(uds),['patdeid','VISIT','t_Opiate300','outcome']][:25]

,patdeid,VISIT,t_Opiate300,outcome
8714,706,0,1.0,1.0
8715,706,1,1.0,0.0
8716,706,2,1.0,0.0
8717,706,3,1.0,0.0
8718,706,4,0.0,0.0
8719,706,5,0.0,0.0
8720,706,6,0.0,0.0
8721,706,7,0.0,0.0
8722,706,8,0.0,0.0
8723,706,9,1.0,0.0


We will save the outcome in a separate dataframe for each unique patient ID and merge later, once remaining tables have been transformed.

In [22]:
def create_outcome_df(df):
    """
    Create a separate dataframe containing unique patient IDs and their corresponding outcomes.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the patient IDs and outcomes.

    Returns:
    pandas.DataFrame: The separate dataframe with unique patient IDs and outcomes.
    """
    outcome_df = df.loc[:, ['patdeid', 'outcome']].drop_duplicates(subset='patdeid')

    # drop outcome from original dataframe
    
    df = df.drop(columns='outcome')

    return df, outcome_df
    


In [23]:
# create target DF and drop from uds dataset
uds, target = create_outcome_df(uds)

# evaluate target value counts and shape
print(f'Shape of target dataframe: {target.shape}')
print('\n')
print('Value counts for target column')
display(target.outcome.value_counts())
print('\n')
print('Confirm outcome columns was dropped from uds dataframe')
display(uds[:5])

Shape of target dataframe: (1917, 2)


Value counts for target column


outcome
0.0    1620
1.0     297
Name: count, dtype: int64



Confirm outcome columns was dropped from uds dataframe


,patdeid,VISIT,t_Propoxyphene,t_Amphetamines,t_alcohol,alc_result,urine_test,refuse_reason,other_reason,urine_temp,supervised,t_Cannabinoids,t_Benzodiazepines,t_Methadone,t_Oxycodone,t_Cocaine,t_Methamphetamine,t_Opiate300
0,1,0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,0.0,1.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
# flatten dataframe for uds

# define parameters
start = 0 # start at zero reflects the baseline assessment week
stop = 24 # relflects the final week of the study
step = 1 # merge will apply for each week

uds = flatten_dataframe(uds,start,stop,step)

# review the first few rows and shape
print(f'Shape of uds dataframe: {uds.shape}')
uds[:5]

Shape of uds dataframe: (1917, 401)


,patdeid,t_Propoxyphene_0,t_Amphetamines_0,t_alcohol_0,alc_result_0,urine_test_0,refuse_reason_0,other_reason_0,urine_temp_0,supervised_0,...,other_reason_24,urine_temp_24,supervised_24,t_Cannabinoids_24,t_Benzodiazepines_24,t_Methadone_24,t_Oxycodone_24,t_Cocaine_24,t_Methamphetamine_24,t_Opiate300_24
0,1,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,...,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,1.0,...,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,3,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,1.0,...,NaN,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,4,0.0,0.0,1.0,0.0,1.0,NaN,NaN,1.0,0.0,...,NaN,99.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
list(uds.columns)

['patdeid',
 't_Propoxyphene_0',
 't_Amphetamines_0',
 't_alcohol_0',
 'alc_result_0',
 'urine_test_0',
 'refuse_reason_0',
 'other_reason_0',
 'urine_temp_0',
 'supervised_0',
 't_Cannabinoids_0',
 't_Benzodiazepines_0',
 't_Methadone_0',
 't_Oxycodone_0',
 't_Cocaine_0',
 't_Methamphetamine_0',
 't_Opiate300_0',
 't_Propoxyphene_1',
 't_Amphetamines_1',
 't_alcohol_1',
 'alc_result_1',
 'urine_test_1',
 'refuse_reason_1',
 'other_reason_1',
 'urine_temp_1',
 'supervised_1',
 't_Cannabinoids_1',
 't_Benzodiazepines_1',
 't_Methadone_1',
 't_Oxycodone_1',
 't_Cocaine_1',
 't_Methamphetamine_1',
 't_Opiate300_1',
 't_Propoxyphene_2',
 't_Amphetamines_2',
 't_alcohol_2',
 'alc_result_2',
 'urine_test_2',
 'refuse_reason_2',
 'other_reason_2',
 'urine_temp_2',
 'supervised_2',
 't_Cannabinoids_2',
 't_Benzodiazepines_2',
 't_Methadone_2',
 't_Oxycodone_2',
 't_Cocaine_2',
 't_Methamphetamine_2',
 't_Opiate300_2',
 't_Propoxyphene_3',
 't_Amphetamines_3',
 't_alcohol_3',
 'alc_result_3',
 

## Feature Engineering
We will create 2 more features for this dataset:
- Total negative tests - we will count the total number of negative opiate test per patient over 24 weeks
- Days since use - we will count the total number of consecutive negative opiate tests per patient over 24 weeks

In [26]:
def uds_features(df):
    """
    Creates features (columns) from opiate test data, listed as follows:
    1) 'tnt' - Total Negative tests - counts total negative tests per patient
    2) 'dsu' - Days since last use - counts number of consecutive weeks of negative tests
    3) 'responder' - If patient meets predefined abstinence window,the final 4 concsecutive weeks of negative tests, then 1 else 0

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the opiates data.

    Returns:
    pandas.DataFrame: The processed DataFrame.
    """
    # create df for opiates tests
    opiates = df.loc[:, ['patdeid'] + [col for col in df.columns if 't_Opiate300' in col]]

    # remove the prefix from the column names
    opiates.columns = opiates.columns.str.replace('t_Opiate300_', '')

    # null values will be treated as positive urine tests and filled with 1.0
    opiates = opiates.fillna(1.0)

    # create column tnt (total negative tests) counts total negative tests for each patient
    opiates['tnt'] = (opiates.iloc[:, 1:] == 0.0).astype(int).sum(axis=1)

    # create column for days since last use (dsu) for each patient
    # counts number of consecutive weeks of negative tests
    opiates['dsu'] = None

    for i in opiates.iloc[:, 1:26]:
        opiates['dsu'] = np.where(opiates[i] == 0.0, opiates['dsu'] + 1, 0)

    # create column 'responder'
    # observe the number in columns 21 - 24 if the sum is equal to zero then value in responder column is 1.0 else 0.0
    opiates['responder'] = np.where((opiates.iloc[:, 21:25].sum(axis=1) == 0), 1.0, 0.0)

    return opiates


In [ ]:
# execute uds features function
opiates = uds_features(uds)

# review shape and first 5 rows
print(f'Shape of opiates dataframe: {opiates.shape}')
opiates[:5]

### Urine drug screen data successfully transformed

## Transform Self Reported use table

In [ ]:
# define parameters
sru_cols = ['VISIT','TFB002A','TFB003A'	'TFB001A','TFB001B','TFB011A','TFB010A','TFB009A','TFB004A','TFB005A','TFB006A','TFB007A','TFB008A','patdeid']

new_columns = {'TFB001B':'alc_qty','TFB002A':'sru_cannabis','TFB003A':'sru_cocaine','TFB001A':'sru_alcohol','TFB011A':'sru_other','TFB010A':'sru_oxycodone','TFB009A':'sru_methadone','TFB004A':'sru_amphetamine','TFB005A':'sru_methamphetamine','TFB006A':'sru_opiates','TFB007A':'sru_benzodiazepines','TFB008A':'sru_propoxyphene'}

# execute clean function
sru = clean_df(sru, sru_cols, new_columns)

print(f'Shape of sru dataframe: {sru.shape}')
sru[:5]

### Aggregate columns
The self reported use surveys were entered manually with multiple rows per patient.  We will aggregate the survey results and reduce the number of rows in the dataset

In [ ]:
index_cols = ['patdeid','VISIT']
agg_dict = {col:'sum' for col in sru.columns if col not in ['patdeid','VISIT','total_visits']}

# execute aggregation
sru = agg_df(sru, index_cols, agg_dict)

print(f'New shape for aggregated dataframe, reduced from 100518 rows to {sru.shape[0]}')
sru[:5]

Now we will flatten the SRU dataset creating columns for all results, reducing rows to one row per patient

In [ ]:
# flatten dataframe for sru

# define parameters
start = 0 # start at zero reflects the baseline assessment week
stop = 24 # relflects the final week of the study
step = 4 # SRU is only administered every 4 weeks

sru = flatten_dataframe(sru,start,stop,step)

# review the first few rows and shape
print(f'Shape of sru dataframe: {sru.shape}')
sru[:5]

In [ ]:
# observe sru columns in long form
list(sru.columns)

### Self reported use data successfully transofrmed

## Transform the Demo dataset

In [ ]:
# define parameters
new_cols = ['patdeid','DEM002','DEM003A','DEM004A','DEM004B','DEM004C','DEM004D','DEM004E']

# rename cols
rename_cols = {'DEM002':'Sex','DEM003A':'Ethnicity','DEM004A':'race_amer_ind','DEM004B':'race_Asian','DEM004C':'race_Black','DEM004D':'race_pacific_isl','DEM004E':'race_White'}

# execute aggregation
demo = clean_df(demo, new_cols, rename_cols)

# imput nulls with 0.0
demo = demo.fillna(0.0)

print(f'New shape for demo df {demo.shape}')
demo[:5]


In [ ]:

def clean_columns(df, cols):
    """
    Create dummy variables for the given columns in the given DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the columns to be dummified.
    cols (list): A list of column names to be dummified.

    Returns:
    pandas.DataFrame: The DataFrame with the dummy variables added.
    """
    for col in cols:
        df[col] = np.where(df[col]>0.0, 1.0, 0.0)
    return df

In [ ]:
# define parameters
clean_cols = ['race_amer_ind','race_Asian','race_Black','race_pacific_isl','race_White']

# execute function
demo = clean_columns(demo, clean_cols)

demo

### All tables have been appropriately cleaned, with new features created and are ready for merge

In [ ]:
# set parameters for merge

# add medication, urine drug screen, self reported use, demographics and target dataframes to list
dfs = [med, uds, sru, demo, total_visits, medication, tnt_dsu, target]

# merge dataframes
new_df = merge_dfs(dfs)

# review shape and columns
print(f'Shape of new dataframe: {new_df.shape}')
list(new_df.columns)

### Tables Successfully Merged

In [ ]:
# export new df to csv
new_df.to_csv('../notebooks.v2/data/merged_data.csv', index=False)